In [20]:
import pandas as pd
import json
from numpyencoder import NumpyEncoder

In [14]:
Avatar=pd.read_csv("../csvs/Avatar.csv")
Partida=pd.read_csv("../csvs/Partida.csv")
Deseado=pd.read_csv("../csvs/Deseado.csv")
Biblioteca=pd.read_csv("../csvs/Biblioteca.csv")
Usuario=pd.read_csv("../csvs/Usuario.csv")
Productora=pd.read_csv("../csvs/Productora.csv")
Saga=pd.read_csv("../csvs/Saga.csv")
Juego=pd.read_csv("../csvs/Juego.csv")

# Generacion de los jsons

### Partidas de los jugadores

In [15]:
# convert date columns to unix timestamps
date_columns = ['FECHAINICIO', 'FECHAFIN']

for df in [Partida]:
    for col in date_columns:
        df[col] = pd.to_datetime(df[col]).astype(int)# convert to unix timestamp in seconds

# create nested json structure
productoras = []

for productora_id, productora_group in Productora.groupby('IDPRODUCTORA'):
    productora_data = {
        'nombre_productora': productora_group['NOMBRE'].iloc[0],
        'sagas': []
    }
    
    for saga_id, saga_group in Saga[Saga['IDPRODUCTORA'] == productora_id].groupby('IDSAGA'):
        saga_data = {
            'nombre_saga': saga_group['NOMBRE'].iloc[0],
            'juegos': []
        }
        
        for juego_id, juego_group in Juego[Juego['IDSAGA'] == saga_id].groupby('IDJUEGO'):
            juego_data = {
                'titulo_juego': juego_group['TITULO'].iloc[0],
                'partidas': []
            }
            
            for _,partida_group in Partida[Partida['IDJUEGO'] == juego_id].groupby('IDJUEGO'):
                
                for i in range(partida_group.shape[0]):
                
                    idUsuario = partida_group['IDUSUARIO'].iloc[i]
                    fecha_ini = partida_group['FECHAINICIO'].iloc[i]
                    fecha_fin = partida_group['FECHAFIN'].iloc[i]
                
                    partida_data = {
                        'fechaini': fecha_ini,
                        'fechafin': fecha_fin,
                        'duracion': abs(fecha_ini - fecha_fin),
                        'usuario': 
                        {
                            'avatar':Avatar[Avatar['IDUSUARIO'] == idUsuario]['NOMBRE'].iloc[0],
                            'username':Usuario[Usuario['IDUSUARIO'] == idUsuario]['NOMBREUSUARIO'].iloc[0]
                        }
                    }
                    juego_data['partidas'].append(partida_data)
            
            saga_data['juegos'].append(juego_data)
        
        productora_data['sagas'].append(saga_data)
    
    productoras.append(productora_data)

# convert to json
json_data = productoras

### Compras

In [17]:
# convert date columns to unix timestamps
date_columns = ['FECHAINICIO', 'FECHAFIN']

for df in [Partida]:
    for col in date_columns:
        df[col] = pd.to_datetime(df[col]).astype(int) // 10**9  # convert to unix timestamp in seconds

# create nested json structure
productoras = []

for productora_id, productora_group in Productora.groupby('IDPRODUCTORA'):
    productora_data = {
        'nombre_productora': productora_group['NOMBRE'].iloc[0],
        'sagas': []
    }
    
    for saga_id, saga_group in Saga[Saga['IDPRODUCTORA'] == productora_id].groupby('IDSAGA'):
        saga_data = {
            'nombre_saga': saga_group['NOMBRE'].iloc[0],
            'juegos': []
        }
        
        for juego_id, juego_group in Juego[Juego['IDSAGA'] == saga_id].groupby('IDJUEGO'):
            juego_data = {
                'titulo_juego': juego_group['TITULO'].iloc[0],
                'ventas': []
            }
            
            for _,biblioteca_group in Biblioteca[Biblioteca['IDJUEGO'] == juego_id].groupby('IDJUEGO'):
                
                for i in range(biblioteca_group.shape[0]):
                
                    idUsuario = biblioteca_group['IDUSUARIO'].iloc[i]
                    idJuego = biblioteca_group['IDJUEGO'].iloc[i]
                    
                    ventas_data = {
                        'fecha_compra': biblioteca_group['FECHAADQUISICION'].iloc[i],
                        'modo_pago':biblioteca_group['MODOADQUISICION'].iloc[i],
                        'precio':biblioteca_group['PRECIOCOMPRA'].iloc[i],
                        'descuento':biblioteca_group['DESCUENTO'].iloc[i],
                        'juego':Juego[Juego['IDJUEGO'] == idJuego]['TITULO'].iloc[0],
                        'usuario': Usuario[Usuario['IDUSUARIO'] == idUsuario]['NOMBREUSUARIO'].iloc[0]
                        
                    }
                    juego_data['ventas'].append(ventas_data)
            
            saga_data['juegos'].append(juego_data)
        
        productora_data['sagas'].append(saga_data)
    
    productoras.append(productora_data)

# convert to json
json_data_compras = productoras

### Guardado

In [21]:
with open('../jsons/jsosns_partidas.json', 'w+') as file:json.dump(json_data, fp = file,separators=(', ', ': '), ensure_ascii=False,cls=NumpyEncoder)

In [22]:
with open('../jsons/jsosns_compras.json', 'w+') as file:json.dump(json_data_compras, fp = file,separators=(', ', ': '), ensure_ascii=False,cls=NumpyEncoder)